In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162993 sha256=42cfd6de135999476a12ba43db92e4b03d07762db4c1adaa84807f3347dcd021
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


### Automatic cross-validation

In [2]:
from surprise import Dataset, SVD
from surprise.model_selection import cross_validate

data = Dataset.load_builtin("ml-100k")
algo = SVD()

cross_validate(algo, data, measures = ["RMSE"], cv=10, verbose=True)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k
Evaluating RMSE of algorithm SVD on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    0.9288  0.9282  0.9221  0.9360  0.9283  0.9185  0.9386  0.9264  0.9319  0.9297  0.9288  0.0056  
Fit time          1.89    1.94    2.72    1.96    1.97    2.04    1.94    2.59    2.00    1.98    2.10    0.28    
Test time         0.08    0.07    0.28    0.07    0.08    0.07    0.07    0.12    0.07    0.11    0.10    0.06    


{'test_rmse': array([0.92881194, 0.92816751, 0.922104  , 0.93596753, 0.9283004 ,
        0.9185045 , 0.93857263, 0.92639274, 0.93189815, 0.92966254]),
 'fit_time': (1.8896770477294922,
  1.937110185623169,
  2.7196662425994873,
  1.9574689865112305,
  1.9687485694885254,
  2.035306692123413,
  1.9413673877716064,
  2.586057424545288,
  2.0005342960357666,
  1.9795880317687988),
 'test_time': (0.07640790939331055,
  0.07301092147827148,
  0.2788503170013428,
  0.0681922435760498,
  0.07867646217346191,
  0.07459259033203125,
  0.07301926612854004,
  0.12317514419555664,
  0.0654287338256836,
  0.11285781860351562)}

In [3]:
data

### Train-test split and fit() method

In [4]:
from surprise import accuracy, Dataset, SVD
from surprise.model_selection import train_test_split

data = Dataset.load_builtin("ml-100k")

trainset, testset = train_test_split(data, test_size = 0.25)

algo = SVD()

algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.9445


0.9444615703614132

### Train on a whole trainset and the predict() method

In [5]:
from surprise import Dataset, KNNBasic

data = Dataset.load_builtin("ml-100k")

trainset = data.build_full_trainset()

algo = KNNBasic()
algo.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [7]:
uid = str(196)
iid = str(302)

# true rating r_ui = 4
pred = algo.predict(uid, iid, r_ui=4, verbose=True)

user: 196        item: 302        r_ui = 4.00   est = 4.06   {'actual_k': 40, 'was_impossible': False}


In [9]:
uid = str(-196)
iid = str(-1231302)

# true rating r_ui = 4
pred = algo.predict(uid, iid, r_ui=4, verbose=True)

user: -196       item: -1231302   r_ui = 4.00   est = 3.53   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}


**We can see, was_impossible = True**

In [11]:
uid = str(2006)
iid = str(-1231302)

# true rating r_ui = 4
pred = algo.predict(uid, iid, r_ui=4, verbose=True)

user: 2006       item: -1231302   r_ui = 4.00   est = 3.53   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}


## Use a custom dataset

In [12]:
import os
import pandas as pd

from surprise import BaselineOnly, Dataset, Reader
from surprise.model_selection import cross_validate

file_path = os.path.expanduser("~/.surprise_data/ml-100k/ml-100k/u.data")

In [15]:
cols = ["user", "item", "rating", "timestamp"]
data = pd.read_csv(file_path, sep="\t", names=cols, encoding="latin-1")
data.head()

,user,item,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [16]:
reader = Reader(line_format="user item rating timestamp", sep="\t")
data = Dataset.load_from_file(file_path, reader=reader)

cross_validate(BaselineOnly(), data, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9477  0.9424  0.9354  0.9468  0.9479  0.9440  0.0048  
MAE (testset)     0.7512  0.7449  0.7418  0.7509  0.7526  0.7483  0.0042  
Fit time          0.69    0.56    0.59    0.61    0.63    0.62    0.04    
Test time         0.28    0.16    0.38    0.10    0.32    0.25    0.10    


{'test_rmse': array([0.9476636 , 0.94238246, 0.93537848, 0.94682546, 0.94791891]),
 'test_mae': array([0.75124447, 0.74491042, 0.74176415, 0.75091924, 0.75260362]),
 'fit_time': (0.6945106983184814,
  0.5618622303009033,
  0.5929772853851318,
  0.6106135845184326,
  0.6271908283233643),
 'test_time': (0.2828989028930664,
  0.1559293270111084,
  0.37778806686401367,
  0.09732294082641602,
  0.3231945037841797)}

#### Load data from pandas dataframe

In [17]:
import pandas as pd

from surprise import Dataset, NormalPredictor, Reader
from surprise.model_selection import cross_validate

ratings_dict = {
    "itemID": [1, 1, 1, 2, 2],
    "userID": [9, 32, 2, 45, "user_foo"],
    "rating": [3, 2, 4, 3, 1],
}

df = pd.DataFrame(ratings_dict)

reader = Reader(rating_scale=(1, 5))

data = Dataset.load_from_df(df[["userID", "itemID", "rating"]], reader)

cross_validate(NormalPredictor(), data, cv=2)

{'test_rmse': array([1.21742141, 2.39911629]),
 'test_mae': array([1.14781311, 1.99429149]),
 'fit_time': (0.10336112976074219, 7.319450378417969e-05),
 'test_time': (0.00022745132446289062, 3.62396240234375e-05)}

### Use cross-validation iterators

In [18]:
from surprise import accuracy, Dataset, SVD
from surprise.model_selection import KFold

data = Dataset.load_builtin("ml-100k")

kf = KFold(n_splits = 3)

algo = SVD()

for trainset, testset in kf.split(data):
  algo.fit(trainset)
  predictions = algo.test(testset)
  accuracy.rmse(predictions, verbose=True)

RMSE: 0.9430
RMSE: 0.9460
RMSE: 0.9464


### Tune algorithm witht he grid search CV

In [19]:
from surprise import Dataset, SVD
from surprise.model_selection import GridSearchCV

data = Dataset.load_builtin("ml-100k")

param_grid = {"n_epochs": [5, 10], "lr_all": [0.002, 0.005], "reg_all": [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures = ["rmse", "mae"], cv=3)

gs.fit(data)

print(gs.best_score["rmse"])

print(gs.best_params["rmse"])

0.9636610748673853
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


In [20]:
# get the instance of the with the optimal set of parameters
algo = gs.best_estimator["rmse"]
algo.fit(data.build_full_trainset())